### THE BENZ CASE

In [1]:
import os
import shutil
import pdfplumber
import logging

# === Réduction des messages parasites ===
logging.getLogger("pdfminer").setLevel(logging.ERROR)

# === Fonctions Utilitaires ===

def clean_value(value):
    return ''.join(filter(str.isdigit, value)) if value else ""

def extract_data(text, keyword):
    for line in text.split('\n'):
        if keyword in line:
            parts = line.split()
            for i, part in enumerate(parts):
                if keyword in part and i + 1 < len(parts):
                    return parts[i + 1]
    return ""

def extract_lieferschein_from_text(text):
    return clean_value(extract_data(text, "Lieferschein")) if "Lieferschein" in text else ""

def extract_belegnummer_from_text(text):
    return clean_value(extract_data(text, "Beleg-Nr.")) if "Beleg-Nr." in text else ""

def extract_belegdatum_from_text(text):
    for line in text.split('\n'):
        if "Belegdatum" in line:
            parts = line.strip().split()
            for part in parts:
                if part.count('.') == 2:
                    return part.replace('.', '-')  # Format AAAA-MM-JJ
    return ""

def extract_lieferdatum_from_text(text):
    for line in text.split('\n'):
        if "Lieferdatum" in line:
            parts = line.strip().split()
            for part in parts:
                if part.count('.') == 2:
                    return part.replace('.', '-')
    return ""

# === Chemins ===

lieferschein_folder = r"C:/Users/Mr Al/Desktop/Full/Benz/Lieferung_Benz"
rechnung_folder = r"C:/Users/Mr Al/Desktop/Full/Benz/Rechnung_Benz"
matched_folder = r"C:/Users/Mr Al/Desktop/Full/Benz/matched"
unmatched_folder = r"C:/Users/Mr Al/Desktop/Full/Benz/unmatched"

os.makedirs(os.path.join(unmatched_folder, "Lieferschein"), exist_ok=True)
os.makedirs(os.path.join(unmatched_folder, "Rechnung"), exist_ok=True)
os.makedirs(matched_folder, exist_ok=True)

# === Extraction des fichiers avec leurs valeurs associées ===

lieferschein_map = {}  # {belegnummer: (chemin, belegdatum)}
for f in os.listdir(lieferschein_folder):
    if f.lower().endswith(".pdf"):
        path = os.path.join(lieferschein_folder, f)
        with pdfplumber.open(path) as pdf:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    belegnummer = extract_belegnummer_from_text(text)
                    belegdatum = extract_belegdatum_from_text(text)
                    if belegnummer:
                        lieferschein_map[belegnummer] = (path, belegdatum)
                        break

rechnung_map = {}  # {lieferschein_num: (chemin, lieferdatum)}
for f in os.listdir(rechnung_folder):
    if f.lower().endswith(".pdf"):
        path = os.path.join(rechnung_folder, f)
        with pdfplumber.open(path) as pdf:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    lieferschein_num = extract_lieferschein_from_text(text)
                    lieferdatum = extract_lieferdatum_from_text(text)
                    if lieferschein_num:
                        rechnung_map[lieferschein_num] = (path, lieferdatum)
                        break

# === Comparaison et tri ===

matched_keys = set(lieferschein_map.keys()) & set(rechnung_map.keys())

# Fichiers appariés
for key in matched_keys:
    lieferschein_path, belegdatum = lieferschein_map[key]
    rechnung_path, _ = rechnung_map[key]

    dir_name = f"{key}_{belegdatum}" if belegdatum else key
    match_dir = os.path.join(matched_folder, dir_name)

    os.makedirs(os.path.join(match_dir, "Lieferschein"), exist_ok=True)
    os.makedirs(os.path.join(match_dir, "Rechnung"), exist_ok=True)

    shutil.copy2(lieferschein_path, os.path.join(match_dir, "Lieferschein", os.path.basename(lieferschein_path)))
    shutil.copy2(rechnung_path, os.path.join(match_dir, "Rechnung", os.path.basename(rechnung_path)))

# Fichiers non appariés
for key, (path, _) in lieferschein_map.items():
    if key not in matched_keys:
        shutil.copy2(path, os.path.join(unmatched_folder, "Lieferschein", os.path.basename(path)))

for key, (path, _) in rechnung_map.items():
    if key not in matched_keys:
        shutil.copy2(path, os.path.join(unmatched_folder, "Rechnung", os.path.basename(path)))

print("Classement terminé.")


PermissionError: [WinError 5] Accès refusé: 'C:/Users/Mr Al/Desktop/Full'

### THE WURTH CASE